# Exchanging Solutions Between Two Nonmatching Spherical Surfaces

In [ ]:
import numpy as np
from pydtk2 import *
from pymoab.core import Core
from pymoab.types import MBQUAD, MB_TYPE_DOUBLE, MB_TAG_DENSE
from mpi4py import MPI

In [ ]:
coarse_file = 'meshes/CubSph2_0_0_0_41_55_63_GaussHillSlotCylnd.vtk'
fine_file = 'meshes/CubSph2_0_0_0_73_81_101_GaussHillSlotCylnd.vtk'

In [ ]:
# the next line should be modified for maximum iterations
itmax = 30
# the next line should be modified for output file names
output_common = 'results/DTKresults'
output_fine, output_coarse = output_common + '_fine', output_common + '_coarse'
# the next line indicates the Radial Basis Function Radius,
# if this value is too small, then the local problems may be singular,
# however, if it's too large, then it may take 'forever' to generate the operators
radius = 0.2

In [ ]:
# MOAB stuffs
mdb_fine, mdb_coarse = Core(), Core()

# load files, this will store point_data in tags
mdb_fine.load_file(fine_file)
mdb_coarse.load_file(coarse_file)

# the point data in vtk files are tagged with 'scalars'
field_fine = field_coarse = 'scalars'

In [ ]:
# Now construct two PyDTK2 MOAB managers
fine_manager = DTK2MoabManager(mdb_fine)
coarse_manager = DTK2MoabManager(mdb_coarse)

In [ ]:
# now we push the tag strings into the managers
fine_manager.register_tag(field_fine)
coarse_manager.register_tag(field_coarse)

In [ ]:
# next step is to construct the DTK2 MOAB mappers
fine2coarse = DTK2Mapper(source=fine_manager, target=coarse_manager, r=radius)
coarse2fine = DTK2Mapper(source=coarse_manager, target=fine_manager, r=radius)

# now register coupled flags
fine2coarse.register_coupled_tags(field_fine, field_coarse)
coarse2fine.register_coupled_tags(field_coarse, field_fine)

In [ ]:
# the transfer loop
for it in range(itmax):
    # dump to file
    mdb_coarse.write_file(output_coarse + '%i.vtk' % it)
    # transfer from fine to coarse
    fine2coarse.apply(field_fine, field_coarse)
    
    # dump to file
    mdb_fine.write_file(output_fine + '%i.vtk' % it)
    # transfer from coarse to fine
    coarse2fine.apply(field_coarse, field_fine)

In [ ]:
# write the final solutions
mdb_coarse.write_file(output_coarse + '%i.vtk' % itmax)
mdb_fine.write_file(output_fine + '%i.vtk' % itmax)